In [5]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_pickle('files/cleaned.pkl')


In [7]:

nominal_cols = ['property.subtype', 'Flood zone type', 'Heating type','Kitchen type',
            'Building condition', 'Type of building','As built plan','property.location.postalCode'
            ]

df = pd.get_dummies(data= df, columns=nominal_cols)

In [8]:
# Ordinal encoding
ordinal_cols= ['Energy class']
scale_mapper = {'A':1,
                'B':2,
                'C':3,
                'D':4,
                'E':5,
                'F':6,
                'Not specified': np.nan
                }

df[ordinal_cols] = df[ordinal_cols].replace(scale_mapper)

In [9]:
# Making a dict of all cols with missing values
missing = {col: df[col].isnull().sum() for col in df.columns if df[col].isnull().sum()  != 0}
missing

{'property.location.box': 1322,
 'property.location.latitude': 232,
 'property.location.longitude': 232,
 'property.location.number': 281,
 'property.location.street': 228,
 'property.netHabitableSurface': 137,
 'Address': 381,
 'Armored door': 1396,
 'Bathrooms': 217,
 'Construction year': 710,
 'Double glazing': 633,
 'E-level (primary energy consumption)': 607,
 'Energy class': 699,
 'Facades': 774,
 'Living area': 157,
 'Living room surface': 1197,
 'Planning permission obtained': 809,
 'Toilets': 542,
 'Cadastral income': 1206,
 'Bedroom 1 surface': 1173,
 'Bedroom 2 surface': 1351,
 'Covered parking spaces': 1546,
 'Intercom': 1342,
 'Terrace surface': 996,
 'Basement': 1516,
 'Number of floors': 867,
 'Elevator': 911}

In [10]:
from sklearn.impute import  MissingIndicator
missing_bin = [x for x in missing.keys() if x in binary_cols]
imp = MissingIndicator()
df[missing_bin]= imp.fit_transform(df[missing_bin])

from sklearn.impute import SimpleImputer
missing_ord= [x for x in missing.keys() if x in ordinal_cols]
imp = SimpleImputer(strategy='median')
df[missing_ord]= imp.fit_transform(df[missing_ord])

missing_num = [x for x in missing.keys() if df[x].isnull().sum() != 0]

from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=2)
df[missing_num] = imputer.fit_transform(df[missing_num])


NameError: name 'binary_cols' is not defined